
# Visualising Multiple Linear Regression
**Model:** `sales ~ adverts + airplay`  

To run this notebook you will need these libraries: 
- `statsmodels` (formula API) for fitting, 
- `plotly` for an interactive 3D plot
- `pandas` for data handling
- `numpy` for numerical operations


In [2]:

import pandas as pd
import numpy as np
from pathlib import Path

import statsmodels.formula.api as smf
import plotly.graph_objects as go


The **Album Sales** dataset, adapted from Andy Field’s *Discovering Statistics Using SPSS*, records information from a sample of 200 albums released by various artists and provides data on their commercial success and promotional efforts. The main outcome variable is **sales**, measured in thousands of units sold. Two key predictors are **adverts**, representing the amount of money (in thousands of pounds) spent on advertising the album, **airplay**, which measures how frequently tracks from the album were played on the radio, and **attract** how
attractive people found the band’s image out of 10. Together, these variables illustrate a simple but realistic marketing scenario, allowing students to explore how promotional investment and radio exposure relate to album sales through multiple linear regression.

- adverts
    – Type: Continuous
    – Description: Amount of money spent on advertisement of the album (in thousands of British Pounds).
- sales
    – Type: Continuous
    – Description: Number of albums sold (in thousands).
- airplay
    – Type: Continuous
    – Description: The number of times songs from the album are played on radio the week before release.
- attract
    – Type: Continuous
    – Description: Attractiveness of the band. The mode of attractiveness ratings given by a random sample of the target audience (0= low attractiveness, 10 = high attractiveness)


In [3]:
# Load data
#df = pd.read_csv("./data/Album Sales 2.dat.txt", sep="\t")
#df.head()

In [4]:
df = pd.read_csv("Album Sales 2.dat.txt", sep="\t")
df.head()


,adverts,sales,airplay,attract
0,10.256,330,43,10
1,985.685,120,28,7
2,1445.563,360,35,7
3,1188.193,270,33,7
4,574.513,220,44,5


In [5]:
# Fit model with statsmodels formula API
model = smf.ols("sales ~ adverts + airplay", data=df).fit()

# print the model "receipt"
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  sales   R-squared:                       0.629
Model:                            OLS   Adj. R-squared:                  0.626
Method:                 Least Squares   F-statistic:                     167.2
Date:                Tue, 11 Nov 2025   Prob (F-statistic):           3.55e-43
Time:                        12:00:06   Log-Likelihood:                -1062.2
No. Observations:                 200   AIC:                             2130.
Df Residuals:                     197   BIC:                             2140.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     41.1238      9.331      4.407      0.0

In [6]:
import plotly.graph_objects as go

# Interactive 3D scatter only (no plane, no residuals)
fig = go.Figure()

fig.add_trace(
    go.Scatter3d(
        x=df["adverts"],
        y=df["airplay"],
        z=df["sales"],
        mode="markers",
        marker=dict(size=5, color='royalblue', opacity=0.8),
        name="Observed data"
    )
)

fig.update_layout(
    title="Album Sales – 3D Scatter (adverts, airplay, sales)",
    scene=dict(
        xaxis_title="adverts",
        yaxis_title="airplay",
        zaxis_title="sales",
        aspectmode="cube"
    ),
    margin=dict(l=0, r=0, t=50, b=0)
)

fig.show()


In [7]:

# Extract parameters for plotting the regression plane
intercept = float(model.params["Intercept"])
b_adverts = float(model.params["adverts"])
b_airplay = float(model.params["airplay"])
r2 = float(model.rsquared)

# Build grid for the plane
a_min, a_max = df["adverts"].min(), df["adverts"].max()
p_min, p_max = df["airplay"].min(), df["airplay"].max()

a_grid = np.linspace(a_min, a_max, 60)
p_grid = np.linspace(p_min, p_max, 60)
A, P = np.meshgrid(a_grid, p_grid)
Z = intercept + b_adverts * A + b_airplay * P


In [18]:

# Interactive 3D figure with Plotly
scatter = go.Scatter3d(
    x=df["adverts"],
    y=df["airplay"],
    z=df["sales"],
    mode="markers",
    marker=dict(size=4, opacity=0.8),
    name="Observed"
)

plane = go.Surface(
    x=A, y=P, z=Z,
    opacity=0.5,
    name="Regression plane",
    showscale=False
)

# Residual lines (we'll plot only a sample to keep the scene responsive)
sample_df = df.copy()
if len(sample_df) > 150:
    sample_df = sample_df.sample(150, random_state=42)

residual_traces = []
for _, row in sample_df.iterrows():
    a, p, actual = row["adverts"], row["airplay"], row["sales"]
    fitted = intercept + b_adverts * a + b_airplay * p
    residual_traces.append(go.Scatter3d(
        x=[a, a],
        y=[p, p],
        z=[fitted, actual],
        mode="lines",
        line=dict(width=2),
        showlegend=False
    ))

fig = go.Figure(data=[scatter, plane] + residual_traces)
fig.update_layout(
    title=f"Multiple Linear Regression (statsmodels): sales ~ adverts + airplay<br>R² = {r2:.3f}",
    scene=dict(
        xaxis_title="adverts",
        yaxis_title="airplay",
        zaxis_title="sales",
        aspectmode="cube"
    ),
    margin=dict(l=0, r=0, t=60, b=0),
)

# Render the interactive chart
fig.show()



### Tips
- Rotate to examine the slope with respect to each predictor.
- Toggle traces in the legend to compare plane vs points vs residuals.
- Use `model.summary()` to interpret coefficients, uncertainty, and R².


# Interactions

Now, let's explore how to include interaction terms in our regression model. Interaction terms allow us to investigate whether the effect of one predictor variable on the outcome variable depends on the level of another predictor variable.

We will use the dataset `album_sales_interaction.csv`, which has been modified to include an interaction effect between `adverts` and `airplay`.

In [72]:
# Load data
df = pd.read_csv("./data/album_sales_interaction.csv")
df.head()

,adverts,airplay,sales
0,336.510,39,52652.466467
1,70.922,26,8965.380228
2,377.925,43,63797.265473
3,50.000,29,1449.740806
4,456.897,22,38045.164864


In [73]:
# Fit model with statsmodels formula API
model2 = smf.ols("sales ~ adverts * airplay", data=df).fit()

# print the model "receipt"
print(model2.summary())


                            OLS Regression Results                            
Dep. Variable:                  sales   R-squared:                       0.995
Model:                            OLS   Adj. R-squared:                  0.995
Method:                 Least Squares   F-statistic:                 1.374e+04
Date:                Mon, 10 Nov 2025   Prob (F-statistic):          1.64e-227
Time:                        12:15:45   Log-Likelihood:                -1955.8
No. Observations:                 200   AIC:                             3920.
Df Residuals:                     196   BIC:                             3933.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept        -592.7393   1446.088     

Let's plot the data again.

In [74]:
import plotly.graph_objects as go

# Interactive 3D scatter only (no plane, no residuals)
fig = go.Figure()

fig.add_trace(
    go.Scatter3d(
        x=df["adverts"],
        y=df["airplay"],
        z=df["sales"],
        mode="markers",
        marker=dict(size=5, color='royalblue', opacity=0.8),
        name="Observed data"
    )
)

fig.update_layout(
    title="Album Sales – 3D Scatter (adverts, airplay, sales)",
    scene=dict(
        xaxis_title="adverts",
        yaxis_title="airplay",
        zaxis_title="sales",
        aspectmode="cube"
    ),
    margin=dict(l=0, r=0, t=50, b=0)
)

fig.show()


Let's add the regression plane

In [75]:
model2.params

Intercept         -592.739331
adverts             -0.444900
airplay              5.256536
adverts:airplay      4.015900
dtype: float64

In [76]:
# Extract parameters for plotting the regression plane
intercept = float(model2.params["Intercept"])
b_adverts = float(model2.params["adverts"])
b_airplay = float(model2.params["airplay"])
b_interaction = float(model2.params["adverts:airplay"])
r2 = float(model2.rsquared)

# Build grid for the plane
a_min, a_max = df["adverts"].min(), df["adverts"].max()
p_min, p_max = df["airplay"].min(), df["airplay"].max()

a_grid = np.linspace(a_min, a_max, 60)
p_grid = np.linspace(p_min, p_max, 60)
A, P = np.meshgrid(a_grid, p_grid)
Z = intercept + b_adverts * A + b_airplay * P + b_interaction * (A * P)


In [77]:
# Interactive 3D figure with Plotly
scatter = go.Scatter3d(
    x=df["adverts"],
    y=df["airplay"],
    z=df["sales"],
    mode="markers",
    marker=dict(size=4, opacity=0.8),
    name="Observed"
)

plane = go.Surface(
    x=A, y=P, z=Z,
    opacity=0.5,
    name="Regression plane",
    showscale=False
)

# Residual lines (we'll plot only a sample to keep the scene responsive)
sample_df = df.copy()
if len(sample_df) > 150:
    sample_df = sample_df.sample(150, random_state=42)

residual_traces = []
for _, row in sample_df.iterrows():
    a, p, actual = row["adverts"], row["airplay"], row["sales"]
    fitted = intercept + b_adverts * a + b_airplay * p + b_interaction * (a * p)
    residual_traces.append(go.Scatter3d(
        x=[a, a],
        y=[p, p],
        z=[fitted, actual],
        mode="lines",
        line=dict(width=2),
        showlegend=False
    ))

fig = go.Figure(data=[scatter, plane] + residual_traces)
fig.update_layout(
    title=f"Multiple Linear Regression (statsmodels): sales ~ adverts + airplay<br>R² = {r2:.3f}",
    scene=dict(
        xaxis_title="adverts",
        yaxis_title="airplay",
        zaxis_title="sales",
        aspectmode="cube"
    ),
    margin=dict(l=0, r=0, t=60, b=0),
)

# Render the interactive chart
fig.show()


### ✏️ Your turn

You are the manager of a music production company deciding which of three new bands to promote. Your goal is to predict which band is expected to sell the most albums, using Model 2** — a multiple linear regression model that includes an *interaction* between advertising spend and radio airplay (`sales ~ adverts * airplay`).

The three candidate bands have the following marketing plans:

| Band | Airplay (radio plays) | Adverts (thousands of £) |
| ---- | --------------------- | ------------------------ |
| 1    | 40                    | 100                      |
| 2    | 10                    | 200                      |
| 3    | 30                    | 100                      |

**Task:**
Using your interaction model, calculate the predicted sales for each band and determine which one the company should promote.

In [47]:
### Your code here